# Neural Affect Style Transfer

In [1]:
from numpy import zeros, concatenate, asarray, ones, amax, argmax
from IPython.display import display, HTML

In [2]:
def browser_alert(message):
    display(HTML('<script type="text/javascript">alert("' + message + '");</script>'))
    
def browser_notify(message):
    display(HTML('<script type="text/javascript">var notification=new Notification("' + \
                 'Jupyter Notification",{icon:"http://blog.jupyter.org/content/' + \
                 'images/2015/02/jupyter-sq-text.png",body:"' + message + \
                 '"});</script>'))

In [3]:
browser_notify("test")

## Read Data

In [4]:
dataset_path = "/home/v2john/attr-reviews-dataset/dev.txt"

In [5]:
all_texts = list()
with open(dataset_path) as dataset_file:
    for line in dataset_file:
        text = line.split('\t')[3]
        all_texts.append(text)

In [6]:
len(all_texts)

93703

### Tokenize and build embeddings

In [58]:
from keras.preprocessing.text import Tokenizer

In [59]:
keras_tokenizer = Tokenizer(num_words=1000)

In [60]:
keras_tokenizer.fit_on_texts(all_texts)

In [61]:
len(keras_tokenizer.word_index)

36663

In [62]:
text_sequences = keras_tokenizer.texts_to_sequences(all_texts)

In [63]:
text_sequences[1001]

[7, 65, 237, 9, 3, 7, 10, 135, 5, 18, 58, 222, 15, 78, 204, 21, 62, 48, 66, 7]

## Keras Model

In [64]:
from keras.layers import Input, Dense, RepeatVector, LSTM, Conv1D, Masking, Embedding
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

In [65]:
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 20
VOCAB_SIZE = 1000

In [66]:
x_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', 
                        truncating='post', value=0)

In [67]:
x_train = x_train[:10000]

In [68]:
x_train.shape

(10000, 20)

In [69]:
x_train_rev = list()
for x_vector in x_train:
    x_rev_vector = list()
    for index in x_vector:
        word_vector = zeros(VOCAB_SIZE)
        word_vector[index] = 1
        x_rev_vector.append(word_vector)
    x_train_rev.append(asarray(x_rev_vector))
x_train_rev = asarray(x_train_rev)

In [70]:
x_train_rev.shape

(10000, 20, 1000)

In [71]:
main_input = Input(shape=x_train.shape[1:], dtype='float32', name='main_input')
embed_1 = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, 
                    mask_zero=True, input_length=MAX_SEQUENCE_LENGTH) (main_input)
print(embed_1)
# lstm_1 = Bidirectional(LSTM(EMBEDDING_DIM, return_sequences=True, name='lstm_1'))(embed_1)
# print(lstm_1)
lstm_2 = Bidirectional(LSTM(EMBEDDING_DIM, name='lstm_2'))(embed_1)
print(lstm_2)
repeat_1 = RepeatVector(MAX_SEQUENCE_LENGTH, name='repeat_1')(lstm_2)
print(repeat_1)
lstm_3 = Bidirectional(LSTM(EMBEDDING_DIM, return_sequences=True, name='lstm_3'))(repeat_1)
print(lstm_3)
# lstm_4 = LSTM(EMBEDDING_DIM, return_sequences=True, name='lstm_4')(lstm_3)
# print(lstm_4)
softmax_1 = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(lstm_3)
print(softmax_1)

Tensor("embedding_4/Gather:0", shape=(?, 20, 100), dtype=float32)
Tensor("bidirectional_6/concat_2:0", shape=(?, 200), dtype=float32)
Tensor("repeat_1_3/Tile:0", shape=(?, 20, 200), dtype=float32)
Tensor("bidirectional_7/concat_2:0", shape=(?, ?, 200), dtype=float32)
Tensor("time_distributed_4/Reshape_1:0", shape=(?, 20, 1000), dtype=float32)


In [72]:
model = Model(main_input, softmax_1)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [73]:
model.fit(x_train, x_train_rev, batch_size=32, epochs=50, verbose=1)

Epoch 1/50
10000/10000 [==============================] - 61s - loss: 5.0252 - acc: 0.1289    
Epoch 2/50
10000/10000 [==============================] - 59s - loss: 4.7601 - acc: 0.1488    
Epoch 3/50
10000/10000 [==============================] - 59s - loss: 4.6396 - acc: 0.1600    
Epoch 4/50
10000/10000 [==============================] - 59s - loss: 4.4789 - acc: 0.1799    
Epoch 5/50
10000/10000 [==============================] - 59s - loss: 4.2833 - acc: 0.2029    
Epoch 6/50
10000/10000 [==============================] - 59s - loss: 4.0911 - acc: 0.2210    
Epoch 7/50
10000/10000 [==============================] - 59s - loss: 3.9094 - acc: 0.2429    
Epoch 8/50
10000/10000 [==============================] - 59s - loss: 3.7290 - acc: 0.2657    
Epoch 9/50
10000/10000 [==============================] - 59s - loss: 3.5532 - acc: 0.2882    
Epoch 10/50
10000/10000 [==============================] - 59s - loss: 3.3991 - acc: 0.3083    - ETA: 0s - loss: 3.3996 - acc: 0.3
Epoch 11/50
10

In [74]:
predictions = model.predict(x_train)

In [75]:
word_dict = dict()
for word in keras_tokenizer.word_index:
    rank = keras_tokenizer.word_index[word]
    if rank <= VOCAB_SIZE:
        word_dict[rank] = word

In [76]:
max_prob = 0.3
def sequence_to_str(sequence):
    word_list = list()
    for element in sequence:
        if amax(element) < max_prob:
            continue        
        index = argmax(element) + 1
        word = word_dict[index]
        word_list.append(word)
        
    return word_list

In [83]:
for i in range(len(predictions)):
    predicted_word_list = sequence_to_str(predictions[i])
    actual_len = len(all_texts[i].split())
    print("Actual: " + all_texts[i])
    print("Generated: " + " ".join(predicted_word_list[:actual_len]) + " - " + "\n")

Actual: i loved this book , it was touching

Generated: and really it of book read extremely the - 

Actual: awaome steamy hot read ! loved it !

Generated: family family is really book the the the - 

Actual: loved this book.lisa gardner never disappoints me !

Generated: really it of because escape the the the - 

Actual: i liked it but want to know more

Generated: and your book love into this life 's - 

Actual: campaigning , lying , cold blooded garbage .

Generated: the the the the the the the the - 

Actual: ca n't wait for the next one .

Generated: down for what but i wait have the - 

Actual: enjoyed this book . i would recommend .

Generated: on it of and next by the the - 

Actual: excellent .contniue the story line of tucker series

Generated: back i you definitely was one the the - 

Actual: great book . read everyone of her books

Generated: not of is world was well the the - 

Actual: the writing was disjointed hard to get into

Generated: i hard read this this the the 

Actual: just read the book and look forward to all the other books in this series

Generated: be is i of a novel when this very i he characters story it one - 

Actual: i enjoyed this novel very much . i read all <num> of the series .

Generated: and on it more do and is very out was i one the the the - 

Actual: this book had an unusual story line that kept my attention to the end .

Generated: it of had not you definitely good keep me same this i she the the - 

Actual: very good read . one you can put down but anxious to begin again .

Generated: more great is have n't has an about love this finishing stories the the the - 

Actual: i do n't know what i expected , but the ending was a surprise !

Generated: and time for life could and came love i liked read to else the the - 

Actual: ca n't wait for the next one . awesome reading really , really good stuff

Generated: down for what but i wait have my like like great human the the the - 

Actual: rush and blaire heat it up in this e

Actual: this installment of the carpathians was excellent never a dissapointment , what can i say they are addicting .

Generated: it mind was read read because could could and has action much much the the the the the the - 

Actual: loved it . felt the love story was believable.well written and really lost myself within the story line.loved it

Generated: really book really i i you read like a a like cute i i you definitely really book - 

Actual: if you like michael connelly 's thrillers , you will love this one . do n't miss it .

Generated: can n't enjoyed so so n't are that it have time for fact book the the the the the - 

Actual: i enjoyed reading the bookthe story is interesting and the information about soy brewing are interestingwell written and pelasant

Generated: and on my i you in them them i a they a the the the the the the - 

Actual: i was glad this book had a happy ending but.i did n't like the way it was written .

Generated: and read wish it of recommend to bad like

Actual: it is not what i thought - it was a good book but not james patterson - i was satisfied

Generated: book in with could and lot lot read to great love love reading crazy usually and read the the the - 

Actual: i would have liked it to be longer but it was a good short story would love to read nore

Generated: and next books your book this this cannot love book read to great you you next next this is the - 

Actual: i cried for gideon.this was great love story and u are an exceptional writer . thumbs up awesome job .

Generated: and gives it read not that you a just had favorite things the the the the - 

Actual: took maybe <num> minutes to read this but it made us giggleit 's a fun book for kids to read

Generated: kind author this is it of book to great of but this is the the - 

Actual: you will love the characters the story will grab at your heart and want more could not put it down

Generated: n't n't that i i i you are see a a loved her with an book about the - 

Actual: a

Actual: this is a wonderful set of books i think every one would enjoy these books i really loved the story

Generated: it in to even laugh was characters and through different that know characters and like really really you - 

Actual: well written and kept my interest without a bunch of filler that authors use to fill for a page count

Generated: from they a keep me awesome style to was this but to 've the the the the - 

Actual: do n't waste your money . i could n't even finish this series . boring and absolutely no plot .

Generated: time for clean see men and ca for last it one a a book '' these the the the - 

Actual: rock with me was great would recommend it to my friends to purchase and read for there future books thanks

Generated: graphic book read not next by book this me both this brothers a is is characters already characters chapter - 

Actual: u got to love all the higgins ladies can go about telling a story kristan is good with drama humor

Generated: this end i has you

...

Actual: hmmm what can i say , another good story . enjoyed reading this book . wait for you is definitely a good read .

Generated: could has and action too great of and on it of this but n't are in to great is the - 

Actual: always enjoy reading about what bill gets himself into . i look forward to reading the rest of the books in the series .

Generated: better romance my num could and and when this my i stop was i characters story i one - 

Actual: pretty good books , i like em anyhow & im hooked on the series so ill be back to read the next lot .

Generated: movie great characters and our will i one as set this this is i wait wonderful the the - 

Actual: good story . no the kind of book i usually read , but it was okay . small town marriage , career and affairs

Generated: great you num i take was of and expect is love book book purchase a the the the - 

Actual: this was a recommendation , but very glad i read it ! you never know what will happen at any time or place !

Generate

Actual: good story it took me sometime to get into it after i got it it was good story i hope next book get better

Generated: great you book recommend mystery this this ending and book book read great you and that wait - 

Actual: wonderful , well written and very interesting . i would recommend this book to anyone at all . buy it and enjoy it .

Generated: even they they a more them and next by it of this suspense first first romance book - 

Actual: rd1 & rd2 were so much fun to read . i love andrew and aubrey and absolutely ca n't wait for book <num> .

Generated: am as do line this is a that a a a down for what but out out the the the - 

Actual: a light entertaining read . just perfect for the beach ! i read this book in a few hours , it was great .

Generated: to without is is fast but i hopefully and is it it of to town town book read not - 

Actual: great series . well developed characters . good melding of fact and fiction . only negative is the anti-catholic bias in the book

Actual: this is honestly terrible . nothing was exciting and thecharacters were lame . this is by far the worst patterson i have ever read

Generated: it in in had reader a it in there i usually and books need is the the the - 

Actual: this book kept me not wanting to put it down . loved abby determination . hope to read more stories like this one !

Generated: it of keep loved loved this an book about about is 's character enjoyed it have the - 

Actual: this is a fun read for a relaxing summer weekend . sit back in a lounger , sip a cool drink and enjoy .

Generated: it in to line is but to amazing story to to a romance the the the the the the - 

Actual: once again the author gets started on the action quickly instead of dragging on for fifty pages . the action follows a believable sequence

Generated: funny stories i have will i way i i until to happens the - 

Actual: this book had me right at the beginning . i hope the author does n't make us wait too long for part <num> !

Gene

Actual: whether history or fiction and having visited norway and their north american connections , the subject of the viking is a favorite subject .

Generated: beautiful beautiful a a i deep was i in to deep the the the the the - 

Actual: started out real slow , confusing <num> story lines going at one time.finally about half way the story line cleared and picked up .

Generated: makes author sexy though during never review little i you definitely a - 

Actual: god , what a beautiful story . be prepared to have your heart break and the tears fall and smile through the tears .

Generated: could to to you this this this story a a a a a i - 

Actual: too much bad language for me that over shadowed the story line . the plot was good , but spoiled by the language .

Generated: want do once but loved i you definitely i these read great love there i expecting the - 

Actual: we have always wonder why <num> and not <num> for a bakers dozen . we will read this special christmas story of givi

Actual: such a sad story of young love and the effects of racism . love should be colorblind . worth reading , ends well .

Generated: excellent to you was must a that was disappointed would fast my book from the the the - 

Actual: a great saga that takes you through <num> generations of the same family and the <num> man that made them all a family .

Generated: to not good n't going out was was off enough a i yet start very very - 

Actual: again james patterson and co author write a great mystery novel that you just ca n't put down , love michael bennet charter .

Generated: stories crazy usually usually put bought to not good n't be down for an about that learning the - 

Actual: this has such a great plot ! very original . nice love triangle with just enough tension . love me some dystopian societies !

Generated: it how how to not series series be say say loved way the the the the - 

Actual: fast paced , entertaining read with great unexpected twists and turns ! i will definitel

In [78]:
browser_notify("Sentences generated")

In [79]:
len(model.layers[1].get_weights()[0])

1000

In [80]:
model.layers[1].get_weights()[0][1]

array([ 0.13002115,  0.07832976, -0.07183127, -0.10807424, -0.22804847,
        0.03545106, -0.01636954,  0.13102452, -0.02571931, -0.00872221,
       -0.04388044,  0.28026825, -0.07355396,  0.0964622 ,  0.01210032,
       -0.0866276 , -0.02872628, -0.00330769, -0.05515078, -0.05701992,
       -0.01993107, -0.03503036, -0.06545355, -0.03799373,  0.05211213,
       -0.04690348, -0.06579034,  0.00082185,  0.05728229, -0.10776291,
        0.05728541, -0.02577387,  0.0294317 ,  0.13310108,  0.00333359,
       -0.06210638,  0.1307335 ,  0.15159008, -0.00062763, -0.05636996,
       -0.09896379, -0.01412879,  0.03236286,  0.05364173, -0.1210125 ,
        0.04003581,  0.02525755, -0.01678964,  0.04320347,  0.15219855,
       -0.02910678, -0.00741061, -0.04232468,  0.09357923, -0.15855388,
       -0.25544801, -0.29155746, -0.20727764, -0.19318317, -0.11689574,
        0.03513511, -0.00913067, -0.05426709,  0.00134405,  0.22971578,
        0.14266331, -0.1132793 ,  0.2071881 , -0.07371109, -0.05